In [31]:
from neo4j import GraphDatabase, basic_auth
import json
import csv

In [3]:
# Load JSON data
with open("test_data/Aaron697_Shields502_c3df3ac0-d8a2-4e2c-9673-43f123441901.json", "r") as f:
    json_data = json.load(f)

In [28]:
json_data['entry'][0]['resource']['name']

[{'use': 'official',
  'family': 'Shields502',
  'given': ['Aaron697'],
  'prefix': ['Mr.']}]

In [30]:
def upload_records(tx, records):
    cypher_query = """
    UNWIND $records as record
    CREATE (n:Record)
    SET n.json = apoc.convert.toJson(record)
    """
    tx.run(cypher_query, records=records)

# Connect to Neo4j and upload data
driver = GraphDatabase.driver(
  "bolt://3.238.143.211:7687",
  auth=basic_auth("neo4j", "checker-shift-chips"))

with driver.session(database="neo4j") as session:
    session.execute_write(upload_records, json_data)

print("Data uploaded successfully!")
driver.close()


Data uploaded successfully!


In [32]:
patients_csv_data = []
with open("csv/patients.csv", newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        patients_csv_data.append(row)

In [33]:
patients_csv_data

[{'Id': 'b9c610cd-28a6-4636-ccb6-c7a0d2a4cb85',
  'BIRTHDATE': '2019-02-17',
  'DEATHDATE': '',
  'SSN': '999-65-3251',
  'DRIVERS': '',
  'PASSPORT': '',
  'PREFIX': '',
  'FIRST': 'Damon455',
  'LAST': 'Langosh790',
  'SUFFIX': '',
  'MAIDEN': '',
  'MARITAL': '',
  'RACE': 'white',
  'ETHNICITY': 'nonhispanic',
  'GENDER': 'M',
  'BIRTHPLACE': 'Middleborough  Massachusetts  US',
  'ADDRESS': '620 Lynch Tunnel Apt 0',
  'CITY': 'Springfield',
  'STATE': 'Massachusetts',
  'COUNTY': 'Hampden County',
  'ZIP': '01104',
  'LAT': '42.08038942501558',
  'LON': '-72.48043144917739',
  'HEALTHCARE_EXPENSES': '9039.1645',
  'HEALTHCARE_COVERAGE': '7964.1255'},
 {'Id': 'c1f1fcaa-82fd-d5b7-3544-c8f9708b06a8',
  'BIRTHDATE': '2005-07-04',
  'DEATHDATE': '',
  'SSN': '999-49-3323',
  'DRIVERS': 'S99941126',
  'PASSPORT': '',
  'PREFIX': '',
  'FIRST': 'Thi53',
  'LAST': 'Wunsch504',
  'SUFFIX': '',
  'MAIDEN': '',
  'MARITAL': '',
  'RACE': 'white',
  'ETHNICITY': 'nonhispanic',
  'GENDER': 'F',

In [ ]:
def upload_records(tx, records):
    cypher_query = """
    UNWIND $records as record
    MERGE(:Patient {id: record.Id})
    """
    tx.run(cypher_query, records=records)

# Connect to Neo4j and upload data
driver = GraphDatabase.driver(
  "bolt://3.238.143.211:7687",
  auth=basic_auth("neo4j", "checker-shift-chips"))

with driver.session(database="neo4j") as session:
    session.execute_write(upload_records, patients_csv_data)